In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline

### 1. Read data

In [2]:
df = pd.read_csv('../../data/processed/canomical_data.csv')
df.head()

,p_categories,p_brand,p_day_created,p_sold_quantity,p_original_price,p_discount_rate
0,7,155,659.0,702,528000,49
1,84,198,974.0,12844,799000,46
2,84,192,1372.0,938,209000,0
3,7,155,593.0,10359,473000,50
4,15,145,529.0,2338,106000,16


### 2. Train test split
&#9889; This cell we choose columns X and y to split train test. Test set with size 20% and Train set with size 80% 

In [3]:
X = df.drop('p_sold_quantity', axis='columns')
y = testLabels = df.p_sold_quantity

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

#### Make polynomial regression by pipeline 

In [4]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(StandardScaler(), PolynomialFeatures(degree), LinearRegression(**kwargs))

#### Create paramater grid to use GridSearchCV

In [5]:
param_grid = {
   'polynomialfeatures__degree': [3,4,5], 
   'linearregression__fit_intercept': [True, False], 
   'linearregression__normalize': [True, False]
}

In [6]:
gr_search = GridSearchCV(PolynomialRegression(), 
                         param_grid, 
                         cv=20, 
                         scoring='neg_mean_squared_error', 
                         verbose=3) 

#### Fit grid search to find best score and best param 

In [ ]:
gr_search.fit(X_train, y_train)

In [ ]:
print('Best Score: %s' % gr_search.best_score_)
print('Best Hyperparameters: %s' % gr_search.best_params_)

Best Score: -1.2277947606474625e+19
Best Hyperparameters: {'linearregression__fit_intercept': False, 'linearregression__normalize': True, 'polynomialfeatures__degree': 3}


In [ ]:
poly_grid = PolynomialRegression(degree=3, fit_intercept=False, normalize=True)

### 3. Train model

In [ ]:
poly_grid.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('linearregression',
                 LinearRegression(fit_intercept=False, normalize=True))])

### 4. Predict with model

In [ ]:
y_pred = poly_grid.predict(X_test)

### 5. Evaluate Model Linear regression with polynomial

In [ ]:
from sklearn.metrics import mean_absolute_error
print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

MSE:  1013254.5587644045
RMSE:  1006.6054633094361
